###### setting work path

In [20]:
import os

print(os.getcwd())
os.chdir(r'C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill')
print(os.getcwd())

C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill
C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill


###### loading file

In [21]:
col_name_reg_date = 'reg_date'
col_name_match_id = 'match_id'
col_name_team_no = 'team_no'
col_name_player_id = 'player_id'
col_name_result = 'result'

file_name = '20180711_ratingSim.xlsx'
input_sheet_name = 'input'
output_sheet_name = 'output'

In [22]:
import pandas as pd


df = pd.read_excel(io=file_name, sheet_name=input_sheet_name, header=0)
df

,reg_date,match_id,team_no,player_id,result
0,2018-07-01 11:00:00,1,0,7,0
1,2018-07-02 11:00:00,1,0,6,0
2,2018-07-03 11:00:00,1,1,3,1
3,2018-07-04 11:00:00,1,1,5,1
4,2018-07-05 11:00:00,2,0,6,0
5,2018-07-06 11:00:00,2,1,3,1


In [25]:
len(df)

6

###### trueskill settings

In [23]:
import trueskill as ts
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.stats import norm
pdf = norm.pdf
cdf = norm.cdf
icdf = norm.ppf  # inverse CDF

def get_draw_margin(p, beta, total_players=2):
    """ Compute the draw margin (epsilon) given the draw probability. """
    return icdf((p + 1.0) / 2) * math.sqrt(total_players) * beta

init_mu = 25
init_sigma = init_mu / 3
init_beta = init_sigma / 2
init_gamma = init_sigma / 100
init_draw_probability = 0.1
init_epsilon = get_draw_margin(0.1, init_beta); init_epsilon

env = ts.TrueSkill(mu=init_mu, sigma=init_sigma, beta=init_beta, tau=init_gamma, draw_probability=init_draw_probability, backend='scipy')

###### df transformer

In [14]:
import pandasql as pdsql


def get_transformed_df(df, col_name_reg_date, col_name_match_id, col_name_result):
    query = """
    select
        t2.match_min_reg_date
        , t1.*
    from 
        df as t1
        inner join
        (
            select 
                """ + col_name_match_id + """
                , min(""" + col_name_reg_date + """) as match_min_reg_date
            from 
                df
            group by 
                """ + col_name_match_id + """ 
        ) as t2
        on
            t1.""" + col_name_match_id + """ = t2.""" + col_name_match_id + """
    order by
        t2.match_min_reg_date
        , t1.""" + col_name_match_id + """
        , t1.""" + col_name_result
    return pdsql.sqldf(query, locals())   

In [15]:
# test code

import pandas as pd


dfx = pd.read_excel(io=file_name, sheet_name=input_sheet_name, header=0)
get_transformed_df(dfx, col_name_reg_date, col_name_match_id, col_name_result)

,match_min_reg_date,reg_date,match_id,team_no,player_id,result
0,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,0,7,0
1,2018-07-01 11:00:00.000000,2018-07-02 11:00:00.000000,1,0,6,0
2,2018-07-01 11:00:00.000000,2018-07-03 11:00:00.000000,1,1,3,1
3,2018-07-01 11:00:00.000000,2018-07-04 11:00:00.000000,1,1,5,1
4,2018-07-05 11:00:00.000000,2018-07-05 11:00:00.000000,2,0,6,0
5,2018-07-05 11:00:00.000000,2018-07-06 11:00:00.000000,2,1,3,1


###### slicing

In [18]:
def get_itermatch(df, col_name_reg_date, col_name_match_id, col_name_team_no, col_name_result):
    """
    df: Pandas dataframe

    yielding match : dict {
        result_code0: [player, ...]
        , result_code1: [player, ...]
        , ...
    }
    """

    def append_player(match, player):
        team_no = player[col_name_team_no]
        if match.get(team_no) is None:  # new team encountered
            match[team_no] = [player]
        else:
            match[team_no].append(player)

    last_match_id = None
    match = {}

    df = get_transformed_df(df, col_name_reg_date, col_name_match_id, col_name_result)

    for idx, row in df.iterrows():
        player = row.to_dict()
        this_match_id = player[col_name_match_id]

        if idx == 0 or last_match_id == this_match_id:  # 1st record or same match with last record
            append_player(match, player)
        else:  # new match record started
            yield match
            match = {}
            append_player(match, player)

        last_match_id = this_match_id

    yield match

In [19]:
# test code

dfx = pd.read_excel(io=file_name, sheet_name=input_sheet_name, header=0)

itermatch = get_itermatch(dfx, col_name_reg_date, col_name_match_id, col_name_team_no, col_name_result)
list(itermatch)

[{0: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 7,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-02 11:00:00.000000',
    'result': 0,
    'team_no': 0}],
  1: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 3,
    'reg_date': '2018-07-03 11:00:00.000000',
    'result': 1,
    'team_no': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 5,
    'reg_date': '2018-07-04 11:00:00.000000',
    'result': 1,
    'team_no': 1}]},
 {0: [{'match_id': 2,
    'match_min_reg_date': '2018-07-05 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-05 11:00:00.000000',
    'result': 0,
    'team_no': 0}],
  1: [{'match_id': 2,
    'match_min_reg_date': '2018-07-05 11:00:00.000000',
    'player_id': 3,
    

###### here we go

In [9]:
class RatingStore():
    def __init__(self, mmrtype: str, default_rating=None):
        
        self.store = dict()  # {"playerid":{"score1":val1, "score2":val2}}
        
        # validation for mmrtype
        if mmrtype.lower() not in {'trueskill', 'elo'}:
            raise ValueError('Unknown mmrtype')
        else:
            self.mmrtype = mmrtype.lower()  # 'trueskill', 'elo', 'glicko', ...
        
        if default_rating is None:
            if self.mmrtype == 'trueskill':
                self.default_rating = {'score1':25, 'score2':8.333}
            elif self.mmrtype == 'elo':
                self.default_rating = {'score1':1200}
        
    def get_player_rating(self, player_id):
        # if it's new player then return default value, or return existing rating
        return self.store.get(player_id) or self.default_rating
        
    def update_rating(self, player_id, rating):
        """
        rating: score list
            for trueskill, rating is lengh 2 list
            for elo, rating is length 1 list or scalar
        """
        # validation for rating that will be updated as.
        if isinstance(rating, dict):
            pass
        else:
            raise ValueError('Invalid rating type. this should be dict, likes {"score1":25, "score2":8.333}')
        
        self.store[player_id] = rating
        
    def __repr__(self):
        repr_parts = []
        for p, r in self.store.items():
            repr_parts.append(str(p) + ': ' + str(r))
        return '\n'.join(repr_parts)
        
    

In [10]:
# test code

rating_store = RatingStore('trueskill')
print(rating_store.get_player_rating(1))

rating_store.update_rating(1, {'score1':1, 'score2':2})
rating_store

{'score1': 25, 'score2': 8.333}


1: {'score1': 1, 'score2': 2}

In [11]:
# moved into update_match_result()

def get_rating_groups(match):
    rating_groups = []
    ranks = []

    for i, (result, team_members) in enumerate(match.items()):  # for each team(result) in the match
        rating_groups.append(dict())
        for j, player in enumerate(team_members):  # for each player in the team
            if j == 0:
                ranks.append(player[col_name_result])

            playerid = player[col_name_player_id]

            r = rating_store.get_player_rating(player[col_name_player_id])
            player['rating_before'] = env.Rating(r.get('scroe1'), r.get('score2'))
            rating_groups[i][playerid] = player['rating_before']
    
    return rating_groups, ranks

In [12]:
# test code

match = {0: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 7,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1}],
  1: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 3,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 5,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1}]}

rating_groups_after, ranks = get_rating_groups(match)
rating_groups_after, ranks

([{6: trueskill.Rating(mu=25.000, sigma=8.333),
   7: trueskill.Rating(mu=25.000, sigma=8.333)},
  {3: trueskill.Rating(mu=25.000, sigma=8.333),
   5: trueskill.Rating(mu=25.000, sigma=8.333)}],
 [0, 1])

In [13]:
env.rate(rating_groups_after, ranks)

[{6: trueskill.Rating(mu=28.108, sigma=7.774),
  7: trueskill.Rating(mu=28.108, sigma=7.774)},
 {3: trueskill.Rating(mu=21.892, sigma=7.774),
  5: trueskill.Rating(mu=21.892, sigma=7.774)}]

In [14]:
# moved into update_match_result()
def set_rating_after(match, player_id, rating_after):
    for team_no, team_members in match.items():
        for i, player in enumerate(team_members):
            if player[col_name_player_id] == player_id:
                match[team_no][i]['rating_after'] = rating_after
            else:
                pass

In [15]:
match

{0: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 7,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 6,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1}],
 1: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 3,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 5,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1

In [16]:
# test code

set_rating_after(match, 6, env.Rating(25, 3))

In [17]:
match

{0: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 7,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 6,
   'rating_after': trueskill.Rating(mu=25.000, sigma=3.000),
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1}],
 1: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 3,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 5,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:0

In [18]:
ts.Rating

trueskill.Rating

In [19]:
def update_match_result(rating_store, match, env):
    """

    :param rating_store: instance of RatingStore
    :param match:
    :param env: trueskill environment
    :return: None
    """

    def get_rating_groups(match):
        """
        rate function of trueskill package requires rating_groups and ranks as arg.
        so we build these two things first.

        and we assume player's current rating is always exist in the rating store.
        """
        _rating_groups = []
        _ranks = []

        for i, (result, team_members) in enumerate(match.items()):  # for each team(result) in the match
            _rating_groups.append(dict())
            for j, player in enumerate(team_members):  # for each player in the team
                if j == 0:
                    _ranks.append(player[col_name_result])

                player['rating_before'] = d_rating = rating_store.get_player_rating(player[col_name_player_id])
                _rating_groups[i][player[col_name_player_id]] = env.Rating(d_rating.get('scroe1'), d_rating.get('score2'))
        return _rating_groups, _ranks

    def set_rating_after(match, player_id, rating_after):
        """
        add 'rating_after' key in the match dict
            which having updated player's rating after the match.
        """
        for team_no, team_members in match.items():
            for i, player in enumerate(team_members):
                if player[col_name_player_id] == player_id:
                    match[team_no][i]['rating_after'] = rating_after
                else:
                    pass

    rating_groups, ranks = get_rating_groups(match)
    rating_groups_after = env.rate(rating_groups, ranks)  # calculating updated rating after the match.

    for i, rating_group in enumerate(rating_groups_after):
        for player_id, ts_rating_after in rating_group.items():
            d_rating_after = {"score1": ts_rating_after.mu, "score2": ts_rating_after.sigma}
            set_rating_after(match, player_id, d_rating_after)
            rating_store.update_rating(player_id, d_rating_after)


In [20]:
# test code

itermatch = get_itermatch(dfx, col_name_reg_date, col_name_match_id, col_name_result)
match_history = list(itermatch)

rating_store = RatingStore('trueskill')

for match in match_history:  # for each match(dict those key is teamno.)
    update_match_result(rating_store, match, env)
    

In [21]:
rating_store

7: {'score1': 28.10817756343541, 'score2': 7.774061438362172}
6: {'score1': 29.05101602136367, 'score2': 6.722900982563271}
3: {'score1': 20.94898397863633, 'score2': 6.722900982563271}
5: {'score1': 21.891822436564595, 'score2': 7.774061438362172}

###### exporting result

In [22]:
match_history[0].get(0)

[{'match_id': 1,
  'match_min_reg_date': '2018-07-01 11:00:00.000000',
  'player_id': 7,
  'rating_after': {'score1': 28.10817756343541, 'score2': 7.774061438362172},
  'rating_before': {'score1': 25, 'score2': 8.333},
  'reg_date': '2018-07-01 11:00:00.000000',
  'result': 0,
  'team_no': 0},
 {'match_id': 1,
  'match_min_reg_date': '2018-07-01 11:00:00.000000',
  'player_id': 6,
  'rating_after': {'score1': 28.10817756343541, 'score2': 7.774061438362172},
  'rating_before': {'score1': 25, 'score2': 8.333},
  'reg_date': '2018-07-02 11:00:00.000000',
  'result': 0,
  'team_no': 0}]

In [23]:
dict_list = []
for match in match_history:
    for team in match.values():
        for player in team:
            dict_list.append(player)

In [24]:
df_out = pd.DataFrame(dict_list)

ordered_head = list(df)
for h in list(df_out):
    if h not in set(list(df)):
        ordered_head.append(h)
ordered_head

df_out = df_out[ordered_head]
df_out

,reg_date,match_id,team_no,player_id,result,match_min_reg_date,rating_after,rating_before
0,2018-07-01 11:00:00.000000,1,0,7,0,2018-07-01 11:00:00.000000,"{'score1': 28.10817756343541, 'score2': 7.7740...","{'score1': 25, 'score2': 8.333}"
1,2018-07-02 11:00:00.000000,1,0,6,0,2018-07-01 11:00:00.000000,"{'score1': 28.10817756343541, 'score2': 7.7740...","{'score1': 25, 'score2': 8.333}"
2,2018-07-03 11:00:00.000000,1,1,3,1,2018-07-01 11:00:00.000000,"{'score1': 21.891822436564595, 'score2': 7.774...","{'score1': 25, 'score2': 8.333}"
3,2018-07-04 11:00:00.000000,1,1,5,1,2018-07-01 11:00:00.000000,"{'score1': 21.891822436564595, 'score2': 7.774...","{'score1': 25, 'score2': 8.333}"
4,2018-07-05 11:00:00.000000,2,0,6,0,2018-07-05 11:00:00.000000,"{'score1': 29.05101602136367, 'score2': 6.7229...","{'score1': 28.10817756343541, 'score2': 7.7740..."
5,2018-07-06 11:00:00.000000,2,1,3,1,2018-07-05 11:00:00.000000,"{'score1': 20.94898397863633, 'score2': 6.7229...","{'score1': 21.891822436564595, 'score2': 7.774..."


In [25]:
ws_dict = pd.read_excel(file_name,
                        sheet_name=None)

ws_dict[output_sheet_name] = df_out

with pd.ExcelWriter(file_name,
                    engine='xlsxwriter',
                    datetime_format='yyyy-mm-dd',
                    date_format='yyyy-mm-dd') as writer:

    for ws_name, df_sheet in ws_dict.items():
        df_sheet.to_excel(writer, sheet_name=ws_name, index=False)